In [ ]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [ ]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [3]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

# db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [4]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [5]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [6]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [7]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df


In [8]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권_상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [9]:
import pandas as pd

df = load_to_df('상권-추정유동인구')
# df12 = load_to_df('상권-직장인구')
df1 = df.copy()
df1["상권/배후지"] = 0
# print(df1.head())
df1_1=df1[df1['기준_년_분기_코드']=='2019_1']
df1_2=df1[df1['기준_년_분기_코드']=='2019_2']

In [10]:
week = {'여성' : ['월요일','화요일','수요일','목요일','금요일'],
       '남성' : ['월요일','화요일','수요일','목요일','금요일'],}

weeks = {'여성' : ['토요일','일요일'], '남성':['토요일','일요일'],}

weekend = {'여성' : ['일요일','월요일','화요일','수요일','목요일','금요일','토요일'],
         '남성' : ['일요일','월요일','화요일','수요일','목요일','금요일','토요일'],}
delete_word = '시간대'

In [11]:
def delete_age_by_time(any_df, any_dict, any_word):
    # df1.filter(like='여성').filter(like='토요일')
    for key, values in any_dict.items():
        for value in values:
            any_df[key+'_'+value+'_시간_유동인구수'] = any_df.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = any_df.filter(like=key).filter(like=value).filter(like=any_word).columns
            any_df.drop(del_columns, axis=1, inplace=True)
                    
    return any_df

def delete_day1(any_df, any_dict):
    # df1.filter(like='여성').filter(like='토요일')
    for key, values in any_dict.items():
        for value in values:
            any_df[key+'_주중_시간_유동인구수'] = any_df.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = any_df.filter(like=key).filter(like=value).columns
            any_df.drop(del_columns, axis=1, inplace=True)            
    return any_df

def delete_day2(any_df, any_dict):
    # df1.filter(like='여성').filter(like='토요일')
    for key, values in any_dict.items():
        for value in values:
            any_df[key+'_주말_시간_유동인구수'] = any_df.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = any_df.filter(like=key).filter(like=value).columns
            any_df.drop(del_columns, axis=1, inplace=True)            
    return any_df

In [12]:
pop_time_sex_total = delete_age_by_time(df1, weekend, delete_word)    # 여성, 남성 요일별, 시간대별 컬럼을 요일 인구로 합

In [13]:
# pop_time_sex_total.info()

In [14]:
pop_time_sex_total=pop_time_sex_total.drop(pop_time_sex_total.iloc[:,5:17], axis=1)

In [15]:
# pop_time_sex_total=pop_time_sex_total.set_index('상권_코드')#.filter(like='요일')

In [16]:
# pop_time_sex_total.info()

In [17]:
pop_s_total=pop_time_sex_total.copy()

In [18]:
pop_total=delete_day1(pop_s_total, week )

In [19]:
# pop_total.head()

In [20]:
pop_total = delete_day2(pop_total, weeks)

In [21]:
# pop_total.info()

In [22]:
# 비율 구하기
def ratio(df_col1,df_col2):
    total = df_col1+df_col2
    total2 = df_col1/total*100
    return total2

In [23]:
pop_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2990 entries, 0 to 2989
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   기준_년_분기_코드      2990 non-null   object
 1   상권_코드           2990 non-null   object
 2   총_유동인구_수        2990 non-null   int64 
 3   남성_유동인구_수       2990 non-null   int64 
 4   여성_유동인구_수       2990 non-null   int64 
 5   월요일_유동인구_수      2990 non-null   int64 
 6   화요일_유동인구_수      2990 non-null   int64 
 7   수요일_유동인구_수      2990 non-null   int64 
 8   목요일_유동인구_수      2990 non-null   int64 
 9   금요일_유동인구_수      2990 non-null   int64 
 10  토요일_유동인구_수      2990 non-null   int64 
 11  일요일_유동인구_수      2990 non-null   int64 
 12  상권/배후지          2990 non-null   int64 
 13  여성_주중_시간_유동인구수  2990 non-null   int64 
 14  남성_주중_시간_유동인구수  2990 non-null   int64 
 15  여성_주말_시간_유동인구수  2990 non-null   int64 
 16  남성_주말_시간_유동인구수  2990 non-null   int64 
dtypes: int64(15), object(2)
memory usage: 397.2+ KB


In [24]:
pop_total['여성_주중시간_유동인구_비율']=ratio(pop_total['여성_주중_시간_유동인구수'],pop_total['여성_주말_시간_유동인구수'])
pop_total['여성_주말시간_유동인구_비율']=ratio(pop_total['여성_주말_시간_유동인구수'],pop_total['여성_주중_시간_유동인구수'])

pop_total['남성_주중시간_유동인구_비율']=ratio(pop_total['남성_주중_시간_유동인구수'],pop_total['남성_주말_시간_유동인구수'])
pop_total['남성_주말시간_유동인구_비율']=ratio(pop_total['남성_주말_시간_유동인구수'],pop_total['남성_주중_시간_유동인구수'])

In [25]:
pop_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2990 entries, 0 to 2989
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   기준_년_분기_코드       2990 non-null   object 
 1   상권_코드            2990 non-null   object 
 2   총_유동인구_수         2990 non-null   int64  
 3   남성_유동인구_수        2990 non-null   int64  
 4   여성_유동인구_수        2990 non-null   int64  
 5   월요일_유동인구_수       2990 non-null   int64  
 6   화요일_유동인구_수       2990 non-null   int64  
 7   수요일_유동인구_수       2990 non-null   int64  
 8   목요일_유동인구_수       2990 non-null   int64  
 9   금요일_유동인구_수       2990 non-null   int64  
 10  토요일_유동인구_수       2990 non-null   int64  
 11  일요일_유동인구_수       2990 non-null   int64  
 12  상권/배후지           2990 non-null   int64  
 13  여성_주중_시간_유동인구수   2990 non-null   int64  
 14  남성_주중_시간_유동인구수   2990 non-null   int64  
 15  여성_주말_시간_유동인구수   2990 non-null   int64  
 16  남성_주말_시간_유동인구수   2990 non-null   int64  
 17  여성_주중시간_유동인구_비

In [26]:
pop_total[pop_total['여성_주중시간_유동인구_비율'].isnull()]   #1001344
pop_total[pop_total['남성_주말시간_유동인구_비율'].isnull()]   #1000026
pop_total=pop_total.fillna(0)  # 비율에서 nan값 발생. 확인해 보니 0을 나눠서 그렇게됨. 0으로 값 대체함.

In [27]:
# pop_total.info()

In [28]:
pop_total['총_주중_유동인구수']=pop_total.iloc[:,4:9].sum(axis=1)
pop_total['총_주말_유동인구수']=pop_total.iloc[:,9:11].sum(axis=1)

In [29]:
pop_total=pop_total.drop(pop_total.iloc[:,5:12], axis=1)

In [30]:
pop_total['주중_유동인구_비율']=ratio(pop_total['총_주중_유동인구수'],pop_total['총_주말_유동인구수'])
pop_total['주말_유동인구_비율']=ratio(pop_total['총_주말_유동인구수'],pop_total['총_주중_유동인구수'])
pop_total['남성_유동인구_비율']=ratio(pop_total['남성_유동인구_수'],pop_total['여성_유동인구_수'])
pop_total['여성_유동인구_비율']=ratio(pop_total['여성_유동인구_수'],pop_total['남성_유동인구_수'])

In [31]:
pop_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2990 entries, 0 to 2989
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   기준_년_분기_코드       2990 non-null   object 
 1   상권_코드            2990 non-null   object 
 2   총_유동인구_수         2990 non-null   int64  
 3   남성_유동인구_수        2990 non-null   int64  
 4   여성_유동인구_수        2990 non-null   int64  
 5   상권/배후지           2990 non-null   int64  
 6   여성_주중_시간_유동인구수   2990 non-null   int64  
 7   남성_주중_시간_유동인구수   2990 non-null   int64  
 8   여성_주말_시간_유동인구수   2990 non-null   int64  
 9   남성_주말_시간_유동인구수   2990 non-null   int64  
 10  여성_주중시간_유동인구_비율  2990 non-null   float64
 11  여성_주말시간_유동인구_비율  2990 non-null   float64
 12  남성_주중시간_유동인구_비율  2990 non-null   float64
 13  남성_주말시간_유동인구_비율  2990 non-null   float64
 14  총_주중_유동인구수       2990 non-null   int64  
 15  총_주말_유동인구수       2990 non-null   int64  
 16  주중_유동인구_비율       2990 non-null   float64
 17  주말_유동인구_비율    

In [50]:
pop_total.columns

Index(['기준_년_분기_코드', '상권_코드', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수', '상권/배후지',
       '여성_주중_시간_유동인구수', '남성_주중_시간_유동인구수', '여성_주말_시간_유동인구수', '남성_주말_시간_유동인구수',
       '여성_주중시간_유동인구_비율', '여성_주말시간_유동인구_비율', '남성_주중시간_유동인구_비율',
       '남성_주말시간_유동인구_비율', '총_주중_유동인구수', '총_주말_유동인구수', '주중_유동인구_비율',
       '주말_유동인구_비율', '남성_유동인구_비율', '여성_유동인구_비율'],
      dtype='object')

In [32]:
df1 = load_to_df('상권배후지-추정유동인구')
df3=df1.copy()
df3["상권/배후지"]=1
df3_1=df1[df1['기준_년_분기_코드']=='2019_1']
df3_2=df1[df1['기준_년_분기_코드']=='2019_2']

In [33]:
def delete_age_by_time(any_df, any_dict, any_word):
    # df1.filter(like='여성').filter(like='토요일')
    for key, values in any_dict.items():
        for value in values:
            any_df['hl_'+key+'_'+value+'_시간_유동인구수'] = any_df.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = any_df.filter(like=key).filter(like=value).filter(like=any_word).columns
            any_df.drop(del_columns, axis=1, inplace=True)
                    
    return any_df

def delete_day1(any_df, any_dict):
    # df1.filter(like='여성').filter(like='토요일')
    for key, values in any_dict.items():
        for value in values:
            any_df['hl_'+key+'_주중_시간_유동인구수'] = any_df.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = any_df.filter(like=key).filter(like=value).columns
            any_df.drop(del_columns, axis=1, inplace=True)            
    return any_df

def delete_day2(any_df, any_dict):
    # df1.filter(like='여성').filter(like='토요일')
    for key, values in any_dict.items():
        for value in values:
            any_df['hl_'+key+'_주말_시간_유동인구수'] = any_df.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = any_df.filter(like=key).filter(like=value).columns
            any_df.drop(del_columns, axis=1, inplace=True)            
    return any_df

In [34]:
pop_time_hl = delete_age_by_time(df3, weekend, delete_word)    # 여성, 남성 요일별, 시간대별 컬럼을 요일 인구로 합

In [35]:
# pop_time_hl.info()

In [36]:
pop_time_hl=pop_time_hl.drop(pop_time_hl.iloc[:,5:17], axis=1)

In [37]:
# pop_time_hl=pop_time_hl.set_index('상권_코드')

In [38]:
pop_s_total_hl=pop_time_hl.copy()

In [39]:
pop_total_hl=delete_day1(pop_s_total_hl, week )

In [40]:
pop_total_hl = delete_day2(pop_total_hl, weeks)

In [41]:
pop_total_hl.head()

,기준_년_분기_코드,상권_코드,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,hl_월요일_유동인구_수,hl_화요일_유동인구_수,hl_수요일_유동인구_수,hl_목요일_유동인구_수,hl_금요일_유동인구_수,hl_토요일_유동인구_수,hl_일요일_유동인구_수,상권/배후지,hl_여성_주중_시간_유동인구수,hl_남성_주중_시간_유동인구수,hl_여성_주말_시간_유동인구수,hl_남성_주말_시간_유동인구수
0,2019_1,1000645,161694,87243,74446,21334,23552,21093,19746,25262,19046,31669,1,12176,13086,13996,17681
1,2019_1,1000646,190850,108281,82567,24107,26263,23440,21755,28149,26770,40371,1,12463,15696,17809,22542
2,2019_1,1000647,238761,118531,120230,28805,31886,32701,27639,36014,25002,56717,1,18183,17776,31284,25399
3,2019_1,1000659,1461213,844363,616850,207306,209675,211512,211242,226887,193423,201172,1,94941,131903,90013,111187
4,2019_1,1000660,1871791,1006595,865199,270634,268851,275138,275837,296301,245161,239879,1,135955,160361,118190,121662


In [42]:
pop_total_hl['hl_여성_주중시간_유동인구_비율']=ratio(pop_total_hl['hl_여성_주중_시간_유동인구수'],pop_total_hl['hl_여성_주말_시간_유동인구수'])
pop_total_hl['hl_여성_주말시간_유동인구_비율']=ratio(pop_total_hl['hl_여성_주말_시간_유동인구수'],pop_total_hl['hl_여성_주중_시간_유동인구수'])

pop_total_hl['hl_남성_주중시간_유동인구_비율']=ratio(pop_total_hl['hl_남성_주중_시간_유동인구수'],pop_total_hl['hl_남성_주말_시간_유동인구수'])
pop_total_hl['hl_남성_주말시간_유동인구_비율']=ratio(pop_total_hl['hl_남성_주말_시간_유동인구수'],pop_total_hl['hl_남성_주중_시간_유동인구수'])

In [43]:
pop_total_hl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   기준_년_분기_코드          2020 non-null   object 
 1   상권_코드               2020 non-null   object 
 2   hl_총_유동인구_수         2020 non-null   int64  
 3   hl_남성_유동인구_수        2020 non-null   int64  
 4   hl_여성_유동인구_수        2020 non-null   int64  
 5   hl_월요일_유동인구_수       2020 non-null   int64  
 6   hl_화요일_유동인구_수       2020 non-null   int64  
 7   hl_수요일_유동인구_수       2020 non-null   int64  
 8   hl_목요일_유동인구_수       2020 non-null   int64  
 9   hl_금요일_유동인구_수       2020 non-null   int64  
 10  hl_토요일_유동인구_수       2020 non-null   int64  
 11  hl_일요일_유동인구_수       2020 non-null   int64  
 12  상권/배후지              2020 non-null   int64  
 13  hl_여성_주중_시간_유동인구수   2020 non-null   int64  
 14  hl_남성_주중_시간_유동인구수   2020 non-null   int64  
 15  hl_여성_주말_시간_유동인구수   2020 non-null   int64  
 16  hl_남성_

In [44]:
pop_total_hl['hl_총_주중_유동인구수']=pop_total_hl.iloc[:,4:9].sum(axis=1)
pop_total_hl['hl_총_주말_유동인구수']=pop_total_hl.iloc[:,9:11].sum(axis=1)

In [45]:
pop_total_hl=pop_total_hl.drop(pop_total_hl.iloc[:,5:12], axis=1)

In [46]:
pop_total_hl['hl_주중_유동인구_비율']=ratio(pop_total_hl['hl_총_주중_유동인구수'],pop_total_hl['hl_총_주말_유동인구수'])
pop_total_hl['hl_주말_유동인구_비율']=ratio(pop_total_hl['hl_총_주말_유동인구수'],pop_total_hl['hl_총_주중_유동인구수'])
pop_total_hl['hl_남성_유동인구_비율']=ratio(pop_total_hl['hl_남성_유동인구_수'],pop_total_hl['hl_여성_유동인구_수'])
pop_total_hl['hl_여성_유동인구_비율']=ratio(pop_total_hl['hl_여성_유동인구_수'],pop_total_hl['hl_남성_유동인구_수'])

In [47]:
new_columns = list(map( lambda s: s.replace('hl_',''), pop_total_hl.columns ))
pop_total_hl.columns = new_columns
pop_total_hl.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,상권/배후지,여성_주중_시간_유동인구수,남성_주중_시간_유동인구수,여성_주말_시간_유동인구수,남성_주말_시간_유동인구수,여성_주중시간_유동인구_비율,여성_주말시간_유동인구_비율,남성_주중시간_유동인구_비율,남성_주말시간_유동인구_비율,총_주중_유동인구수,총_주말_유동인구수,주중_유동인구_비율,주말_유동인구_비율,남성_유동인구_비율,여성_유동인구_비율
0,2019_1,1000645,161694,87243,74446,1,12176,13086,13996,17681,46.523002,53.476998,42.532584,57.467416,160171,44308,78.331271,21.668729,53.957288,46.042712
1,2019_1,1000646,190850,108281,82567,1,12463,15696,17809,22542,41.170058,58.829942,41.048172,58.951828,178132,54919,76.434772,23.565228,56.736775,43.263225
2,2019_1,1000647,238761,118531,120230,1,18183,17776,31284,25399,36.757839,63.242161,41.171975,58.828025,241261,61016,79.814541,20.185459,49.644205,50.355795
3,2019_1,1000659,1461213,844363,616850,1,94941,131903,90013,111187,51.332223,48.667777,54.260973,45.739027,1456585,420310,77.606099,22.393901,57.785073,42.214927
4,2019_1,1000660,1871791,1006595,865199,1,135955,160361,118190,121662,53.495052,46.504948,56.860965,43.139035,1955659,541462,78.316549,21.683451,53.777018,46.222982


In [59]:
pop_total.columns
pop_total_hl.columns
len(pop_total['상권_코드'].unique()) , len(pop_total_hl['상권_코드'].unique())

(1495, 1010)

In [ ]:
pop_total=[['기준_년_분기_코드', '상권_코드', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수', '상권/배후지',
       '여성_주중_시간_유동인구수', '남성_주중_시간_유동인구수', '여성_주말_시간_유동인구수', '남성_주말_시간_유동인구수',
       '여성_주중시간_유동인구_비율', '여성_주말시간_유동인구_비율', '남성_주중시간_유동인구_비율',
       '남성_주말시간_유동인구_비율', '총_주중_유동인구수', '총_주말_유동인구수', '주중_유동인구_비율',
       '주말_유동인구_비율', '남성_유동인구_비율', '여성_유동인구_비율']]

pop_total_hl=[['기준_년_분기_코드', '상권_코드', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수', '상권/배후지',
       '여성_주중_시간_유동인구수', '남성_주중_시간_유동인구수', '여성_주말_시간_유동인구수', '남성_주말_시간_유동인구수',
       '여성_주중시간_유동인구_비율', '여성_주말시간_유동인구_비율', '남성_주중시간_유동인구_비율',
       '남성_주말시간_유동인구_비율', '총_주중_유동인구수', '총_주말_유동인구수', '주중_유동인구_비율',
       '주말_유동인구_비율', '남성_유동인구_비율', '여성_유동인구_비율']]

In [60]:
pop_total.append(pop_total_hl)


,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,상권/배후지,여성_주중_시간_유동인구수,남성_주중_시간_유동인구수,여성_주말_시간_유동인구수,남성_주말_시간_유동인구수,여성_주중시간_유동인구_비율,여성_주말시간_유동인구_비율,남성_주중시간_유동인구_비율,남성_주말시간_유동인구_비율,총_주중_유동인구수,총_주말_유동인구수,주중_유동인구_비율,주말_유동인구_비율,남성_유동인구_비율,여성_유동인구_비율
0,2019_1,1000001,118266,61268,56997,0,9103,10008,7228,6575,55.740616,44.259384,60.350962,39.649038,124690,36772,77.225601,22.774399,51.805691,48.194309
1,2019_1,1000002,246619,147358,99261,0,14616,20191,16578,30739,46.855164,53.144836,39.644610,60.355390,220832,77733,73.964463,26.035537,59.751276,40.248724
2,2019_1,1000003,144480,80073,64407,0,10823,13782,7294,8392,59.739471,40.260529,62.153874,37.846126,147469,45734,76.328525,23.671475,55.421512,44.578488
3,2019_1,1000012,166390,70339,96051,0,15119,10842,11764,9943,56.240003,43.759997,52.162617,47.837383,191471,49276,79.532040,20.467960,42.273574,57.726426
4,2019_1,1000013,65518,33437,32080,0,5415,5668,3836,3871,58.534213,41.465787,59.419226,40.580774,67752,22139,75.371283,24.628717,51.035609,48.964391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2019_2,1000884,1462461,717503,744954,1,111012,106611,105751,108014,51.213537,48.786463,49.673151,50.326849,1556956,436701,78.095480,21.904520,49.061477,50.938523
2016,2019_2,1000885,2026927,1069686,957239,1,149515,170163,102503,103044,59.327112,40.672888,62.283543,37.716457,2219235,559406,79.867640,20.132360,52.773832,47.226168
2017,2019_2,1000931,1401131,711644,689491,1,107431,113583,85254,70984,55.754729,44.245271,61.540254,38.459746,1549722,384699,80.112964,19.887036,50.790538,49.209462
2018,2019_2,1000932,2133743,1164525,969217,1,143145,173387,134094,160684,51.632346,48.367654,51.901243,48.098757,2192382,615846,78.069943,21.930057,54.576655,45.423345


In [ ]:
data=[pop_total, pop_total_hl]
    
def name(df):
    df['분기_ID']=df['기준_년_분기_코드'].astype('str')+'-'+ df['상권_코드'].astype('str')
    return m['분기_ID']

In [ ]:
pop_total.reset_index('상권_코드')
name(pop_total)

In [ ]:
pop_total.merge(pop_total_hl, left_on='상권_코드', right_on='상권_코드')